# Importing Modules

In [90]:
import pandas as pd
import numpy as np

# Data Preprocessing

## Importing Dataset

In [93]:
# Define columns to be used for analysis and reading data from the csv file 

analysis_cols = ["Jurisdiction", 
                "Year", 
                "Fuel", 
                "Quantity"]

energy = pd.read_csv("Data/AES 2024 Table O FY - machine-readable.csv",
                     usecols = analysis_cols
                    )

energy

,Jurisdiction,Year,Fuel,Quantity
0,AUS,1989-90,"Bagasse, wood",750.000
1,AUS,1989-90,Biogas,NaN
2,AUS,1989-90,Black coal,87573.000
3,AUS,1989-90,Brown coal,33594.000
4,AUS,1989-90,Geothermal,NaN
...,...,...,...,...
2080,WA,2022-23,Small-scale solar PV,3159.368
2081,WA,2022-23,Total,44478.213
2082,WA,2022-23,Total non-renewable,36841.251
2083,WA,2022-23,Total renewable,7636.963


In [94]:
energy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2085 entries, 0 to 2084
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Jurisdiction  2085 non-null   object 
 1   Year          2085 non-null   object 
 2   Fuel          2085 non-null   object 
 3   Quantity      1527 non-null   float64
dtypes: float64(1), object(3)
memory usage: 65.3+ KB


In [95]:
energy.describe()

,Quantity
count,1527.000000
mean,22617.454900
std,47413.594407
min,0.039000
25%,419.524500
50%,2964.239000
75%,17069.561500
max,274474.842000


## Handling Missing Values

It can be inferred that the missing values for the Quantity Column means that there was no energy generated, thus the missing
values for the Quantity column will be filled with 0s.

In [98]:
energy["Quantity"] = energy["Quantity"].fillna(0)

energy

,Jurisdiction,Year,Fuel,Quantity
0,AUS,1989-90,"Bagasse, wood",750.000
1,AUS,1989-90,Biogas,0.000
2,AUS,1989-90,Black coal,87573.000
3,AUS,1989-90,Brown coal,33594.000
4,AUS,1989-90,Geothermal,0.000
...,...,...,...,...
2080,WA,2022-23,Small-scale solar PV,3159.368
2081,WA,2022-23,Total,44478.213
2082,WA,2022-23,Total non-renewable,36841.251
2083,WA,2022-23,Total renewable,7636.963


## Converting the year columns from fiscal to calendar 

In [100]:
# define a function to convert the fiscal years to calendar yeear  

def convert_fiscal_year(x):
    x = int(x[0:4]) + 1
    return x

# apply the function to year column

energy["Year"] = energy["Year"].apply(convert_fiscal_year)

energy

,Jurisdiction,Year,Fuel,Quantity
0,AUS,1990,"Bagasse, wood",750.000
1,AUS,1990,Biogas,0.000
2,AUS,1990,Black coal,87573.000
3,AUS,1990,Brown coal,33594.000
4,AUS,1990,Geothermal,0.000
...,...,...,...,...
2080,WA,2023,Small-scale solar PV,3159.368
2081,WA,2023,Total,44478.213
2082,WA,2023,Total non-renewable,36841.251
2083,WA,2023,Total renewable,7636.963


Drop the Total, Total non-renewable and Total renewable columns to reduce the chance of rounding errors

In [102]:
rows = ["Total",
        "Total non-renewable",
        "Total renewable"
       ]

energy = energy.query("Fuel not in @rows")

energy["Fuel"].unique()

array(['Bagasse, wood', 'Biogas', 'Black coal', 'Brown coal',
       'Geothermal', 'Hydro', 'Large-scale solar PV', 'Natural gas',
       'Oil products', 'Other', 'Small-scale solar PV', 'Wind'],
      dtype=object)

In [103]:
# Filter the data to create a DataFrame for Australian energy statistics

aus_energy = energy.query("Jurisdiction == 'AUS'")
aus_energy

,Jurisdiction,Year,Fuel,Quantity
0,AUS,1990,"Bagasse, wood",750.000
1,AUS,1990,Biogas,0.000
2,AUS,1990,Black coal,87573.000
3,AUS,1990,Brown coal,33594.000
4,AUS,1990,Geothermal,0.000
...,...,...,...,...
502,AUS,2023,Natural gas,48865.237
503,AUS,2023,Oil products,4864.397
504,AUS,2023,Other,0.000
505,AUS,2023,Small-scale solar PV,25908.682


In [104]:
# Filter the data to create a DataFrame for Australian energy statistics by state

state_energy = energy.query("Jurisdiction != 'AUS'")

state_energy

,Jurisdiction,Year,Fuel,Quantity
510,NSW,2009,"Bagasse, wood",291.300
511,NSW,2009,Biogas,383.400
512,NSW,2009,Black coal,67650.300
513,NSW,2009,Brown coal,0.000
514,NSW,2009,Geothermal,0.000
...,...,...,...,...
2077,WA,2023,Natural gas,27302.995
2078,WA,2023,Oil products,2197.932
2079,WA,2023,Other,0.000
2080,WA,2023,Small-scale solar PV,3159.368


In [105]:
# Create categories for renewable and non renewable 

renewable = ["Bagasse, wood",
             "Biogas",
             "Geothermal",
             "Hydro",
             "Large-scale solar PV",
             "Small-scale solar PV",
             "Wind"]

nonrenewable = ["Black coal", 
               "Brown coal", 
               "Natural gas", 
               "Oil products",
               "Other"]

In [106]:
# Create a pivot table for Australia

energy_pivot_aus = (aus_energy
                    .pivot_table(index = "Year",
                                 columns = "Fuel",
                                 values = "Quantity")
                   )

energy_pivot_aus

Fuel,"Bagasse, wood",Biogas,Black coal,Brown coal,Geothermal,Hydro,Large-scale solar PV,Natural gas,Oil products,Other,Small-scale solar PV,Wind
Year,,,,,,,,,,,,
1990,750.000,0.000,87573.000,33594.000,0.000,14880.000,0.000,14359.000,3552.000,0.0,0.000,0.000
1991,769.000,0.000,89511.000,36048.000,0.000,16103.000,0.000,10772.000,3396.000,0.0,0.000,0.000
1992,670.000,0.000,94325.100,34559.000,0.000,15768.000,0.000,11633.000,2362.000,0.0,10.900,0.000
1993,670.000,0.000,97872.700,33248.000,0.000,16953.000,0.000,12295.000,2421.000,0.0,13.300,0.000
1994,670.000,0.000,100544.100,34890.000,0.000,16649.000,0.000,12199.000,2320.000,0.0,15.900,4.000
1995,670.000,53.000,102522.100,35832.000,0.000,16239.000,0.000,14913.000,2738.000,0.0,18.900,7.000
1996,670.000,258.000,106089.600,39427.000,0.000,15731.000,0.000,12445.000,2805.000,0.0,23.400,7.000
1997,670.000,295.000,109452.200,41893.000,0.000,16852.000,0.000,11426.000,2175.000,0.0,27.800,7.000
1998,670.000,359.000,116969.500,46633.000,0.000,15733.000,0.000,12934.000,1821.000,0.0,33.500,8.000


In [107]:
# Create the new Total_renewable, Total_nonrenewable and Total_Electricity columns
energy_pivot_aus = (energy_pivot_aus
                    .assign(Total_renewable = energy_pivot_aus[renewable].sum(axis = 1),
                            Total_nonrenewable = energy_pivot_aus[nonrenewable].sum(axis = 1),
                            Total_Electricity = energy_pivot_aus.sum(axis = 1)
                   )
                   )

energy_pivot_aus

Fuel,"Bagasse, wood",Biogas,Black coal,Brown coal,Geothermal,Hydro,Large-scale solar PV,Natural gas,Oil products,Other,Small-scale solar PV,Wind,Total_renewable,Total_nonrenewable,Total_Electricity
Year,,,,,,,,,,,,,,,
1990,750.000,0.000,87573.000,33594.000,0.000,14880.000,0.000,14359.000,3552.000,0.0,0.000,0.000,15630.000,139078.000,154708.000
1991,769.000,0.000,89511.000,36048.000,0.000,16103.000,0.000,10772.000,3396.000,0.0,0.000,0.000,16872.000,139727.000,156599.000
1992,670.000,0.000,94325.100,34559.000,0.000,15768.000,0.000,11633.000,2362.000,0.0,10.900,0.000,16448.900,142879.100,159328.000
1993,670.000,0.000,97872.700,33248.000,0.000,16953.000,0.000,12295.000,2421.000,0.0,13.300,0.000,17636.300,145836.700,163473.000
1994,670.000,0.000,100544.100,34890.000,0.000,16649.000,0.000,12199.000,2320.000,0.0,15.900,4.000,17338.900,149953.100,167292.000
1995,670.000,53.000,102522.100,35832.000,0.000,16239.000,0.000,14913.000,2738.000,0.0,18.900,7.000,16987.900,156005.100,172993.000
1996,670.000,258.000,106089.600,39427.000,0.000,15731.000,0.000,12445.000,2805.000,0.0,23.400,7.000,16689.400,160766.600,177456.000
1997,670.000,295.000,109452.200,41893.000,0.000,16852.000,0.000,11426.000,2175.000,0.0,27.800,7.000,17851.800,164946.200,182798.000
1998,670.000,359.000,116969.500,46633.000,0.000,15733.000,0.000,12934.000,1821.000,0.0,33.500,8.000,16803.500,178357.500,195161.000


In [108]:
# Rename the columns

# Create dictionary
colnames = {'Bagasse, wood':'Biomass (Bagasse, Wood)',
            'Black coal' :'Black Coal',
            'Brown coal':'Brown Coal', 
            'Large-scale solar PV':'Large-scale Solar PV',
            'Natural gas': 'Natural Gas', 
            'Oil products':'Oil Products', 
            'Small-scale solar PV':'Small-scale solar PV',
            'Total_renewable': 'Total Renewable', 
            'Total_nonrenewable': 'Total Non-Renewable',
            'Total_Electricity': 'Total Electricity'}

In [110]:
energy_pivot_aus = energy_pivot_aus.rename(colnames, axis = 1)

In [111]:
energy_pivot_aus.to_csv("data/energy_mix_1990_2023.csv")